<a href="https://colab.research.google.com/github/JosenildoVicente/projetoCriatividadeComputacional/blob/main/projeto_criaComp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Enter site key

In [ ]:
#@title
key = input("Enter key: ")

#Run the program:
  Coloque no site anvil para ativar a função chamada 'callColab' onde a entrada deve ser o arquivo de imagem e a saída será outro arquivo de imagem de mesma extensão!

##Install Components

In [ ]:
#@title
!pip install anvil-uplink

In [4]:
#@title
import anvil
import anvil.server
import anvil.media

# from PIL import Image

from google.colab import files

###Style transfer

In [5]:
#@title
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
import tensorflow_hub as hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [6]:
#@title
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

##Conect to site uplink

In [7]:
#@title
anvil.server.connect(key)

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


##Functions to image process

###style transfer functions

In [8]:
#@title
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

Download images and choose a style image and a content image:

In [9]:
#@title
def file_transform_tf(content_image,content_name, style_image, style_name):
  content_path = tf.keras.utils.get_file(content_name, content_image)
  style_path = tf.keras.utils.get_file(style_name,style_image)
  return [content_path,style_path]

Define a function to load an image and limit its maximum dimension to 512 pixels.

In [10]:
#@title
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

Create a simple function to display an image:

In [11]:
#@title
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [12]:
#@title
def plot_images(content_path,style_path):
  content_image = load_img(content_path)
  style_image = load_img(style_path)

  plt.subplot(1, 2, 1)
  imshow(content_image, 'Content Image')

  plt.subplot(1, 2, 2)
  imshow(style_image, 'Style Image')
  plt.show()
  return [content_image,style_image]

#### Fast Style Transfer using TF-Hub

This tutorial demonstrates the original style-transfer algorithm, which optimizes the image content to a particular style. Before getting into the details, let's see how the [TensorFlow Hub model](https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2) does this:

In [13]:
#@title
def stylize_image(content_image,style_image):
  hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
  print("tipo hub: ",type(hub_model))
  stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
  print("tipo stylized_image:",type(stylize_image))
  img_out = tensor_to_image(stylized_image)
  print("img:",img_out)
  return img_out

###Gerenal function

In [14]:
#@title
def change_image(file):
  folder_name = 'images/'
  try:
    os.mkdir('./'+folder_name)
  except OSError:
    print("pasta já existe!")

  image_uploaded_initial_name = file.name
  image_uploaded_name = folder_name + image_uploaded_initial_name
  image_uploaded = anvil.media.write_to_file(file, image_uploaded_name)
  image_style_uploaded = 'https://i.pinimg.com/474x/8c/34/c2/8c34c2e4b43f10f3edb6343c66ead399--linolium-printmaking.jpg'
  image_style_uploaded_name = 'xilo.jpg'
  
  content_path = image_uploaded_name
  style_path = tf.keras.utils.get_file(image_style_uploaded_name,image_style_uploaded)
  
  content_image,style_image = plot_images(content_path,style_path)

  image_stylized = stylize_image(content_image,style_image)

  result_image_name = folder_name + 'new_' + image_uploaded_initial_name
  image_stylized.save(result_image_name)

  new_img_show = load_img(result_image_name)
  imshow(new_img_show, 'New Image')
  plt.show()

  result = anvil.media.from_file(result_image_name)
  
  return result

In [15]:
#@title
@anvil.server.callable
def callColab(file):
  new_image = change_image(file)
  return new_image


##Up connection to the site

In [ ]:
#@title
anvil.server.wait_forever()